In [63]:
# Initial imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from pathlib import Path

%matplotlib inline

In [64]:
# Load Data 
housing_df = pd.read_csv(Path("../data/housing_data_encoded.csv"), index_col="Date", parse_dates=True, infer_datetime_format=True)
housing_df

,Identifier,Bedroom,Price,Average_Market_Days,RegionName_Crosby,RegionName_Houston,RegionName_Pasadena,RegionName_Porter,State_TX,Metro_Houston-The Woodlands-Sugar Land,CountyName_Harris County,CountyName_Montgomery County
Date,,,,,,,,,,,,
2010-01-31,2,1,67653.0,133.0,0,1,0,0,1,1,1,0
2010-01-31,2,2,89995.0,133.0,0,1,0,0,1,1,1,0
2010-01-31,2,3,116641.0,133.0,0,1,0,0,1,1,1,0
2010-01-31,2,4,167849.0,133.0,0,1,0,0,1,1,1,0
2010-01-31,2,5,303323.0,133.0,0,1,0,0,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
2020-02-29,223,1,79380.0,100.0,0,0,1,0,1,1,1,0
2020-02-29,223,2,121517.0,100.0,0,0,1,0,1,1,1,0
2020-02-29,223,3,159204.0,100.0,0,0,1,0,1,1,1,0


In [65]:
#housing_df.reset_index(drop=True, inplace=True)


In [66]:
housing_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1220 entries, 2010-01-31 to 2020-02-29
Data columns (total 12 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   Identifier                              1220 non-null   int64  
 1   Bedroom                                 1220 non-null   int64  
 2   Price                                   1151 non-null   float64
 3   Average_Market_Days                     1030 non-null   float64
 4   RegionName_Crosby                       1220 non-null   int64  
 5   RegionName_Houston                      1220 non-null   int64  
 6   RegionName_Pasadena                     1220 non-null   int64  
 7   RegionName_Porter                       1220 non-null   int64  
 8   State_TX                                1220 non-null   int64  
 9   Metro_Houston-The Woodlands-Sugar Land  1220 non-null   int64  
 10  CountyName_Harris County                12

In [67]:
housing_df= housing_df.dropna()



In [87]:
housing_price_2020= housing_df.loc['2020']
#len(housing_price_2020)
housing_price_2020.corr()

,Identifier,Bedroom,Price,Average_Market_Days,RegionName_Crosby,RegionName_Houston,RegionName_Pasadena,RegionName_Porter,State_TX,Metro_Houston-The Woodlands-Sugar Land,CountyName_Harris County,CountyName_Montgomery County
Identifier,1.000000,-0.250057,-0.229307,0.727167,0.728833,-0.602216,-0.205000,0.588002,NaN,NaN,-0.588002,0.588002
Bedroom,-0.250057,1.000000,0.929843,-0.133892,-0.138409,0.415227,-0.271830,-0.138409,NaN,NaN,0.138409,-0.138409
Price,-0.229307,0.929843,1.000000,-0.164463,-0.172721,0.484220,-0.383951,-0.047817,NaN,NaN,0.047817,-0.047817
Average_Market_Days,0.727167,-0.133892,-0.164463,1.000000,0.978446,-0.322455,-0.232859,-0.085323,NaN,NaN,0.085323,-0.085323
RegionName_Crosby,0.728833,-0.138409,-0.172721,0.978446,1.000000,-0.333333,-0.218218,-0.111111,NaN,NaN,0.111111,-0.111111
RegionName_Houston,-0.602216,0.415227,0.484220,-0.322455,-0.333333,1.000000,-0.654654,-0.333333,NaN,NaN,0.333333,-0.333333
RegionName_Pasadena,-0.205000,-0.271830,-0.383951,-0.232859,-0.218218,-0.654654,1.000000,-0.218218,NaN,NaN,0.218218,-0.218218
RegionName_Porter,0.588002,-0.138409,-0.047817,-0.085323,-0.111111,-0.333333,-0.218218,1.000000,NaN,NaN,-1.000000,1.000000
State_TX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Metro_Houston-The Woodlands-Sugar Land,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [69]:
# Calculate & display 2020  SMA
#housing_df['SMA'] = housing_df['Price'].loc['2020'].rolling(50).mean()
#df[['Last', 'SMA']].plot(title='BTC-USD Daily Price & 50 Day SMA: 2016-Present',figsize=(10,6))

In [88]:
# Creating the X and y sets
X = housing_df.drop(columns=["Price", "Identifier", "State_TX", "Metro_Houston-The Woodlands-Sugar Land"]).values
y = housing_df["Price"].values

In [89]:
# Creating training, validation, and testing sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

#X_train, X_val, y_train, y_val = train_test_split(X_test, y_test, random_state=0)

In [90]:
X_train[0]

array([ 3., 99.,  0.,  0.,  1.,  0.,  1.,  0.])

In [91]:
# Import the StandardScaler from sklearn
from sklearn.preprocessing import StandardScaler

# Scale the data
scaler = StandardScaler().fit(X_train)
X = scaler.transform(X_train)

In [93]:
# Build and train the Deep Learning Model

# Import Keras modules for model creation
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [94]:
# Model set-up
number_input_features = 8
hidden_nodes_layer1 = 50
hidden_nodes_layer2 = 30

In [95]:
# Define the LSTM RNN model
model = Sequential()

# Layer 1
model.add(
    Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Layer 2
model.add(Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
model.add(Dense(1, activation="linear"))

In [96]:
# Compile the model
model.compile(
    loss= "mean_squared_error",
    optimizer="adam",
    metrics= ['accuracy']
   
)

In [97]:
# Summarize the model
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 50)                450       
_________________________________________________________________
dense_13 (Dense)             (None, 30)                1530      
_________________________________________________________________
dense_14 (Dense)             (None, 1)                 31        
Total params: 2,011
Trainable params: 2,011
Non-trainable params: 0
_________________________________________________________________


In [99]:
# Training the model
batch_size = 10
epochs = 1000
training_history = model.fit(
    X_train,
    y_train,
    epochs=epochs,
    batch_size=batch_size,
    verbose=1,
)

ep - loss: 804928704.0000 - accuracy: 0.0000e+00
Epoch 826/1000
78/78 [==============================] - 0s 2ms/step - loss: 800599168.0000 - accuracy: 0.0000e+00
Epoch 827/1000
78/78 [==============================] - 0s 2ms/step - loss: 801083456.0000 - accuracy: 0.0000e+00
Epoch 828/1000
78/78 [==============================] - 0s 2ms/step - loss: 803927296.0000 - accuracy: 0.0000e+00
Epoch 829/1000
78/78 [==============================] - 0s 2ms/step - loss: 799467328.0000 - accuracy: 0.0000e+00
Epoch 830/1000
78/78 [==============================] - 0s 2ms/step - loss: 801394816.0000 - accuracy: 0.0000e+00
Epoch 831/1000
78/78 [==============================] - 0s 2ms/step - loss: 799750400.0000 - accuracy: 0.0000e+00
Epoch 832/1000
78/78 [==============================] - 0s 2ms/step - loss: 798429952.0000 - accuracy: 0.0000e+00
Epoch 833/1000
78/78 [==============================] - 0s 2ms/step - loss: 801183616.0000 - accuracy: 0.0000e+00
Epoch 834/1000
78/78 [=================

In [80]:
housing_df.dtypes

Identifier                                  int64
Bedroom                                     int64
Price                                     float64
Average_Market_Days                       float64
RegionName_Crosby                           int64
RegionName_Houston                          int64
RegionName_Pasadena                         int64
RegionName_Porter                           int64
State_TX                                    int64
Metro_Houston-The Woodlands-Sugar Land      int64
CountyName_Harris County                    int64
CountyName_Montgomery County                int64
dtype: object

In [81]:
y_train

array([ 99407.,  81927., 100907.,  49212.,  64958., 106299.,  89795.,
       179445., 128711.,  85898., 118832.,  64626.,  77129., 107210.,
        75147.,  71813., 125669., 163679., 163244.,  94147., 290524.,
       126468., 105561.,  50283., 378108.,  92183., 193420., 114616.,
       191297., 115283., 107056.,  97789.,  94465., 373130., 235604.,
       157223., 293166., 389004.,  77070., 337713.,  76737., 134826.,
       171767., 103071.,  81074., 233793., 291775.,  64759., 387842.,
       105929., 110757., 223174., 157161.,  97965.,  96411., 390638.,
        55704., 119186., 116422.,  84886., 297672., 138302.,  92947.,
        85323., 107146., 399976.,  78094.,  66197., 143914., 235069.,
       123792., 108983., 303716., 362116., 234391., 116185., 108656.,
        48807.,  99636.,  88608., 158942.,  55945., 142246., 292821.,
        48692., 296905.,  65187.,  71537.,  94545., 159204., 132441.,
       123866., 323600.,  64045., 162823., 301892., 133380., 291667.,
       388934., 3536